In [25]:
# Importing Libraries
from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM

In [2]:
# Initialize the model
llm = Ollama(model='tinyllama')

C:\Users\weare\AppData\Local\Temp\ipykernel_25556\775502121.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model='tinyllama')


In [6]:
# Generate answers to a question
question = "When did India get Independence?"
response = llm.invoke(question)
print(response)

India got Independence on August 15, 1947, from British colonial rule.


In [8]:
# Generate answers to a question
question = "Who is Sachin Tendulkar"
response = llm.invoke(question)
print(response)

Sachin Tendulkar is the founder and CEO of Mobify, an artificial intelligence (AI) technology company headquartered in Pune, India. He has been a tech entrepreneur since 2014 and has developed several successful startups in the areas of e-commerce, finance, and customer service.

Tendulkar has a Bachelor's degree in Computer Science from Pune University, an MBA in Marketing & Strategy from IIM Bangalore, and has completed an Executive Program in Entrepreneurship at the Harvard Business School. He is also a regular speaker and mentor on startup founding, investment, and growth.

Tendukkar's expertise lies in developing AI-driven products and services to enhance customer experience and increase efficiency in business operations. Mobify has successfully leveraged machine learning and natural language processing (NLP) technologies to develop innovative solutions for clients like Walmart, Amazon, and Alibaba. He is also a published author of several books on entrepreneurship and startup suc

## Implementing RAG for custom data

In [15]:
# LLM (later, can use Ollama or HuggingFace)
# import openai  # only if you use OpenAI API

# Vector store
from langchain_community.vectorstores import FAISS

# Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.embeddings import OpenAIEmbeddings  # optional

# PDF loader
from langchain_community.document_loaders import PyPDFLoader

# Text splitter
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter





ModuleNotFoundError: No module named 'langchain.document_loaders'

In [17]:
!pip install --upgrade langchain langchain-text-splitters langchain-community


In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader


In [21]:

pdf_loader = PyPDFLoader("Attention Is All You Need.pdf")
documents = pdf_loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

print("Total Chunks:", len(chunks))
print(chunks[0].page_content[:500])



Total Chunks: 52
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz 


In [23]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Build FAISS vector store
db = FAISS.from_documents(documents=chunks, embedding=embeddings)

# Create retriever
retriever = db.as_retriever()


C:\Users\weare\AppData\Local\Temp\ipykernel_25556\3079478810.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [26]:
llm = OllamaLLM(model="tinyllama")


In [28]:
from langchain_core.prompts import PromptTemplate

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Use the following context to answer the question:

Context:
{context}

Question:
{question}

Answer:
"""
)


In [32]:
# Fetch relevant docs directly from FAISS
docs = db.similarity_search(query, k=5)  # k = number of top relevant chunks


# Format the context
context = "\n\n".join([d.page_content for d in docs])

# Prepare prompt
final_prompt = prompt.format(context=context, question=query)

# Query LLM
response = llm.invoke(final_prompt)
print(response)



The self-attention mechanism in encoder layers of pre-trained language models (e.g., BERT or GPT) involves a recurrent layer that applies a sequence of attention weights to each input token, attended by a set of previously computed keys and values based on the positions of other tokens in the sequence. This process is repeated for multiple layers until the input sequence is fully processed. The output of the self-attention mechanism is a weighted sum of the attended representations, allowing the model to generate contextualized embeddings for each token.

The self-attention mechanism can be seen as a form of "attentional embedding" in which attention weights are applied to each input token to selectively attend to its relevant context. This can improve the representation of each token by enhancing its "topicality", making it easier for the model to process and understand different parts of the sequence. By allowing the self-attention mechanism to attend to the entire sequence, the mode

## Updated ask_question function

In [33]:
def ask_question(query):
    # Retrieve relevant chunks
    docs = db.similarity_search(query, k=5)
    context = "\n\n".join([d.page_content for d in docs])
    
    # Prepare prompt
    final_prompt = prompt.format(context=context, question=query)
    
    # Query LLM
    response = llm.invoke(final_prompt)
    return response

# Test
query = "Explain self-attention mechanism"
answer = ask_question(query)
print(answer)


Self-attention is a type of attention mechanism found in various layers of deep neural networks (DNNs), which maps one variable-length sequence of symbol representations (such as an input word or sentence) to another variable-length sequence of equal length (called the "query", "key", and "value" vectors). Self-attention is designed to compute the attention score for each possible combination of query, key, and value vectors based on the distribution of the underlying variable-length sequence.

The self-attention mechanism can be understood as a way to leverage information across multiple time steps in a sequence by taking into account all possible positions where the values might occur. This is achieved by mapping each time step in the sequence to a set of "self-attention" operations, where each operation takes an output vector from the previous time step and a query and key vector for that time step and computes the attention score between them.

The total computational complexity of

### Chat history - bot



### Importing Libraries 

In [1]:
!pip install --upgrade langchain langchain-community langchain-text-splitters ollama sentence-transformers faiss-cpu


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate


### Load PDF and Split into Chunks

In [3]:
pdf_loader = PyPDFLoader("Attention Is All You Need.pdf")
documents = pdf_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

print("Total chunks:", len(chunks))


Total chunks: 52


### Create Embeddings and FAISS Vector Store

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(documents=chunks, embedding=embeddings)


C:\Users\weare\AppData\Local\Temp\ipykernel_15984\3635374491.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


## Initialize Ollama LLM

In [11]:
llm = OllamaLLM(model="tinyllama",gpu=False)


### Define PromptTemplate

In [6]:
prompt = PromptTemplate(
    input_variables=["chat_history", "context", "question"],
    template="""
The following is a conversation history between a user and an AI assistant:
{chat_history}

Use the context below to answer the latest question.
Context:
{context}

Question:
{question}

Answer:
"""
)


### Chat History Setup

In [7]:
chat_history = []  # list of strings: "User: ...\nAI: ..."


### Helper to combine documents

In [8]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


### Ask Question Function with Chat History

In [9]:
def ask_question(query):
    # Retrieve relevant chunks
    docs = db.similarity_search(query, k=5)
    context = format_docs(docs)
    
    # Combine chat history
    history_text = "\n".join(chat_history)
    
    # Format prompt
    final_prompt = prompt.format(chat_history=history_text, context=context, question=query)
    
    # Query LLM
    response = llm.invoke(final_prompt)
    
    # Update chat history
    chat_history.append(f"User: {query}")
    chat_history.append(f"AI: {response}")
    
    return response


### Example Usage

In [12]:
query1 = "Explain self-attention mechanism"
answer1 = ask_question(query1)
print(answer1)

query2 = "How does positional encoding work?"
answer2 = ask_question(query2)
print(answer2)


The self-attenition mechanism involves mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum of these vectors, and the resulting vector produces an attention mechanism in which the importance or "weight" of each vector in the output is proportional to its contribution towards the overall goal of mapping the query to the desired output.

To explain this more specifically, let us consider the case where we have a sequence of symbols (x1, ..., xn) that we want to map to a sequence of symbols (y1, ..., ym), each representing a word or a concept. We can think of the query as a single symbol that represents the desired output, and the vector representation of this query is a fixed-length vector of size n with all its elements set to 1.

The weights for each of these vectors in the output are determined by the context surrounding the query. Specifically, they depend on the history of symbo